In [9]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import random 

In [10]:
# Load the model
model = keras.models.load_model("model_ann_3layer")

In [11]:
#Inputs
s = 50 #size of the grid
N = 1000 #size of population
M = round(N * 0.07) #Number of infectious population
Et = 2 #Number of days staying exposed
It = 21 #Number of days staying infectious
Mt = 5 #Number of daily movements
D = 200 #Number of days
death_rate = 100
expose_rate = 5

#Initialization
S = N - M #Susceptible population
E = 0 #Exposed population
I = M #Number of infectious population 
R = 0 #Recovered population
P = S + E + I + R #Total population
economy = 0 #Daily economic transaction

In [12]:
# Create a virtual environment actions
def reset():
    global P, M, It, s
    dummy_array = np.zeros(shape=(P,8))
    df = pd.DataFrame(dummy_array,columns=['x','y','Day','Susceptible','Exposed','Infectious','Recovered','GG'])
    df = df.astype({'x':int,'y':int,'Day':int,'Susceptible':bool,'Exposed':int,'Infectious':int,'Recovered':bool,'GG':bool})
    df['Susceptible'] = True
    #Appending infectious population in 
    dfupdate=df.sample(M)
    dfupdate['Infectious'] = np.random.randint(1,It, size=len(dfupdate))
    dfupdate['Susceptible'] = False
    df.update(dfupdate)
    update_list = dfupdate.index.tolist() 
    #Dispersing people randomly among grid
    df['x'] = np.random.randint(0,s, size=len(df))
    df['y'] = np.random.randint(0,s, size=len(df))

    return df

    
def coor_around(p, df):
    return [(df.loc[p, 'x'] + a, df.loc[p, 'y'] + b) for a in range(-1,2) for b in range(-1, 2)]

def one_day(df, action = 0):

    # start_time = time.time()
    global P, M, It, S, death_rate, expose_rate
    policy_match = {0: 1, 1:0.75, 2:0.25} # assign action to policy
    moves_under_policy = int(round(Mt * policy_match[action], 0))

    df_infectious = df.loc[(df['Infectious'] > 0)]
    df_infectious = df_infectious[['x', 'y']]

    for mt in range(moves_under_policy):
        for index, person in df.iterrows():

            if not person['GG']:  # If the person is not dead

                new_move_x = random.choice(range(-1, 2))
                new_move_y = random.choice(range(-1, 2))

                person['x'] = max(min(person['x'] + new_move_x, s), 0)
                person['y'] = max(min(person['y'] + new_move_y, s), 0)

                df.iat[index, 0] = int(person['x'])
                df.iat[index, 1] = int(person['y'])

                if index in df_infectious.index:  # assigning whats in person (row) to df_infectious at the correct index
                    df_infectious.at[index, 'x'] = person['x']
                    df_infectious.at[index, 'y'] = person['y']

                df.at[index, 'Day'] = day + 1  # updating the day counter

                if (person['Infectious'] > 0) and (person['Recovered'] == False):  # If a person is in infectious state
                    if person['Infectious'] - random.choice(range(0, 7)) >= It:  # If the infectious days are completed
                        if random.choice(range(0, death_rate)) > (
                                death_rate - 2):  # If the person dies(with probability distribution 1:4)
                            df.at[index, 'Infectious'] = 0
                            if index in df_infectious.index:
                                df_infectious.drop([index])

                            df.at[index, 'GG'] = True  # Kill the person
                        else:  # If the person survives
                            df.at[index, 'Infectious'] = 0
                            if index in df_infectious.index:
                                df_infectious.drop([index])
                            df.at[index, 'Recovered'] = True  # Recover the person
                    elif mt + 1 == Mt:
                        df.at[index, 'Infectious'] = person['Infectious'] + 1  # Increase the infectious day counter
                elif (person['Exposed'] > 0) and (person['Infectious'] == 0):  # If a person is in exposed state
                    if (person['Exposed'] - random.choice(
                            range(0, 2))) >= Et:  # If the person has reached the exposed day limit?  7
                        df.at[index, 'Exposed'] = 0
                        df.at[
                            index, 'Infectious'] = 1  # Increase the infectious day counter, now the person is infectious
                        df_infectious.append(person)
                    elif mt + 1 == Mt:
                        df.at[index, 'Exposed'] = person['Exposed'] + 1  # Increase the exposed day counter

                elif person['Susceptible']:  # If the person is in susceptible state

                    x_temp = int(person['x'])
                    df_xtemp = df_infectious[['x']].to_numpy()

                    if (x_temp in df_xtemp)or ((x_temp - 1) in df_xtemp) or ((x_temp + 1) in df_xtemp):

                        y_temp = int(person['y'])
                        df_ytemp = df_infectious[['y']].to_numpy()
                        if (y_temp in df_ytemp) or ((y_temp - 1) in df_ytemp) or ((y_temp + 1) in df_ytemp):
                            if random.choice(range(0, expose_rate)) > (expose_rate - 2):
                                df.at[index, 'Exposed'] = 1
                                df.at[index, 'Susceptible'] = False
    # print("--- %s seconds ---" % (time.time() - start_time))
    
    return df # time.time() - start_time #


def economy_gain(df):
    economy_gain = len(df[(df.GG == False) & (df.Infectious == 0)]) * round(random.uniform(0.8,1), 2)
    return economy_gain

def current_state(df):
    inf = len(df.loc[df['Infectious'] > 0])
    exposed = len(df.loc[df['Exposed'] > 0]) 
    recovered = len(df.loc[df['Recovered'] == True])
    sus = len(df.loc[df['Susceptible'] == True])
    gg = df.loc[df['GG'] == True].GG.count()
    
    return np.array([recovered,sus, exposed, inf, gg])



In [13]:
# Use the agent to make decisions
df = reset()
economy = 0

for day in range(0, D+1):
    state = current_state(df)
    state =tf.reshape(state, [1, 5])
    prediction = model.predict(state, steps = 1)
    action_by_agent = np.argmax(prediction)
    df = one_day(df, action = action_by_agent)
    gain = economy_gain(df)
    economy += gain
    print(f"Day {day}: take action {action_by_agent}, total_reward: {economy}. {prediction}")


Day 0: take action 2, total_reward: 846.3000000000001. [[7922.235 7822.677 8066.618]]
Day 1: take action 2, total_reward: 1599.6000000000001. [[7028.767 6944.888 7148.979]]
Day 2: take action 2, total_reward: 2418.0. [[6428.3164 6377.0273 6521.3813]]
Day 3: take action 2, total_reward: 3338.7. [[6001.749  6020.159  6052.5117]]
Day 4: take action 1, total_reward: 4110.599999999999. [[5759.054  5859.8384 5771.417 ]]
Day 5: take action 1, total_reward: 4873.199999999999. [[5180.078 5574.848 5065.095]]
Day 6: take action 1, total_reward: 5728.799999999999. [[4894.801 5392.247 4852.658]]
Day 7: take action 1, total_reward: 6602.999999999999. [[4748.6763 5297.5864 4747.473 ]]
Day 8: take action 1, total_reward: 7384.199999999999. [[4670.95   5247.2344 4691.524 ]]
Day 9: take action 1, total_reward: 8304.9. [[4642.969  5229.108  4671.3823]]
Day 10: take action 1, total_reward: 9141.9. [[4618.0967 5212.9956 4653.478 ]]
Day 11: take action 1, total_reward: 10053.3. [[4599.443  5200.9116 4640.05

KeyboardInterrupt: 